示例代码中以NLP中的核心任务——主题建模为例，展示Gensim库中的各种功能
+ 主题建模 —— 从文档中提取其语义主题
+ 主题 —— 指的是一组出现在一起代表某种语义的词

# 语料库和字典

Gensim中没有对于对原始文本进行预处理的方式和接口进行强制约束，可以使用不同的方法对原始文本（根据任务的需求）进行预处理：分词、处理停用词、词干提取、大小写转换等等。Gensim本身提供了simple_preprocess方法进行预处理，方法会对文本进行分词和标准化之后返回tokne序列

Gensim本身提供了downloader模块，通过该模块可以下载数据集或者模型

In [1]:
import gensim.downloader as api

dataset_info = api.info("text8")
print(dataset_info)


{'num_records': 1701, 'record_format': 'list of str (tokens)', 'file_size': 33182058, 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py', 'license': 'not found', 'description': 'First 100,000,000 bytes of plain text from Wikipedia. Used for testing purposes; see wiki-english-* for proper full Wikipedia datasets.', 'checksum': '68799af40b6bda07dfa47a32612e5364', 'file_name': 'text8.gz', 'read_more': ['http://mattmahoney.net/dc/textdata.html'], 'parts': 1}


In [2]:
import gensim
from gensim.utils import simple_preprocess
 
# open the text file as an object
doc = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]
 
# preprocess the file to get a list of tokens
tokenized =[]
for sentence in doc:
  # the simple_preprocess function returns a list of each sentence
  tokenized.append(simple_preprocess(sentence, deacc = True))
    
print(tokenized)

[['human', 'machine', 'interface', 'for', 'lab', 'abc', 'computer', 'applications'], ['survey', 'of', 'user', 'opinion', 'of', 'computer', 'system', 'response', 'time'], ['the', 'eps', 'user', 'interface', 'management', 'system'], ['system', 'and', 'human', 'system', 'engineering', 'testing', 'of', 'eps'], ['relation', 'of', 'user', 'perceived', 'response', 'time', 'to', 'error', 'measurement'], ['the', 'generation', 'of', 'random', 'binary', 'unordered', 'trees'], ['the', 'intersection', 'graph', 'of', 'paths', 'in', 'trees'], ['graph', 'minors', 'iv', 'widths', 'of', 'trees', 'and', 'well', 'quasi', 'ordering'], ['graph', 'minors', 'survey']]


对于经过预处理的文档，Gensim可以通过Dictionary类根据语料构建字典，字典对语料中每个单词赋予一个整数索引，根据这个索引可以使用Dictionary的方法将句子转换为词袋模型的向量

In [3]:
from gensim import corpora
my_dictionary = corpora.Dictionary(tokenized)
print(my_dictionary)
print(my_dictionary.token2id) # token到其id的map
print(my_dictionary.doc2bow(tokenized[0])) # 将文本转换为词袋模型的向量

Dictionary<41 unique tokens: ['abc', 'applications', 'computer', 'for', 'human']...>
{'abc': 0, 'applications': 1, 'computer': 2, 'for': 3, 'human': 4, 'interface': 5, 'lab': 6, 'machine': 7, 'of': 8, 'opinion': 9, 'response': 10, 'survey': 11, 'system': 12, 'time': 13, 'user': 14, 'eps': 15, 'management': 16, 'the': 17, 'and': 18, 'engineering': 19, 'testing': 20, 'error': 21, 'measurement': 22, 'perceived': 23, 'relation': 24, 'to': 25, 'binary': 26, 'generation': 27, 'random': 28, 'trees': 29, 'unordered': 30, 'graph': 31, 'in': 32, 'intersection': 33, 'paths': 34, 'iv': 35, 'minors': 36, 'ordering': 37, 'quasi': 38, 'well': 39, 'widths': 40}
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]


Gensim提供了不同的方法存储和读取字典

In [4]:
# 通过Dictionary默认存储方式 保存和打开字典
my_dictionary.save('my_dict.dict')
load_dict = corpora.Dictionary.load('my_dict.dict')
print(load_dict.token2id) # token到其id的map
print(load_dict.doc2bow(tokenized[0])) # 将文本转换为词袋模型的向量

# 将字典用文本文件读取
from gensim.test.utils import get_tmpfile
tmp_fname = get_tmpfile('dictionary')
my_dictionary.save_as_text(tmp_fname)
load_dict = corpora.Dictionary.load_from_text(tmp_fname)
print(load_dict.token2id) # token到其id的map
print(load_dict.doc2bow(tokenized[0])) # 将文本转换为词袋模型的向量

{'abc': 0, 'applications': 1, 'computer': 2, 'for': 3, 'human': 4, 'interface': 5, 'lab': 6, 'machine': 7, 'of': 8, 'opinion': 9, 'response': 10, 'survey': 11, 'system': 12, 'time': 13, 'user': 14, 'eps': 15, 'management': 16, 'the': 17, 'and': 18, 'engineering': 19, 'testing': 20, 'error': 21, 'measurement': 22, 'perceived': 23, 'relation': 24, 'to': 25, 'binary': 26, 'generation': 27, 'random': 28, 'trees': 29, 'unordered': 30, 'graph': 31, 'in': 32, 'intersection': 33, 'paths': 34, 'iv': 35, 'minors': 36, 'ordering': 37, 'quasi': 38, 'well': 39, 'widths': 40}
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
{'abc': 0, 'and': 18, 'applications': 1, 'binary': 26, 'computer': 2, 'engineering': 19, 'eps': 15, 'error': 21, 'for': 3, 'generation': 27, 'graph': 31, 'human': 4, 'in': 32, 'interface': 5, 'intersection': 33, 'iv': 35, 'lab': 6, 'machine': 7, 'management': 16, 'measurement': 22, 'minors': 36, 'of': 8, 'opinion': 9, 'ordering': 37, 'paths': 34, 'perceived': 23, 

Gensim支持通过Matrix Market格式保存和读取语料库

In [5]:
from gensim.corpora import MmCorpus
from gensim.test.utils import get_tmpfile

BoW_corpus =[my_dictionary.doc2bow(doc, allow_update = True) for doc in tokenized]
print(BoW_corpus) # 将文本转换为词袋模型的向量

output_fname = get_tmpfile("BoW_corpus.mm") 
# 存储语料库
MmCorpus.serialize(output_fname, BoW_corpus)
# 读取语料库
load_corpus = MmCorpus(output_fname)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)], [(5, 1), (12, 1), (14, 1), (15, 1), (16, 1), (17, 1)], [(4, 1), (8, 1), (12, 2), (15, 1), (18, 1), (19, 1), (20, 1)], [(8, 1), (10, 1), (13, 1), (14, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1)], [(8, 1), (17, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)], [(8, 1), (17, 1), (29, 1), (31, 1), (32, 1), (33, 1), (34, 1)], [(8, 1), (18, 1), (29, 1), (31, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)], [(11, 1), (31, 1), (36, 1)]]


# Gensim中使用TF-IDF模型

TF-IDF — Term Frequency(词频)-Inverse Document Frequency(逆文档频率)，是一种用于信息检索和数据挖掘的加权技术。

字词的重要性与它在文件中出现的次数成正比增加，但同时会随着在语料库中出现的频率成反比下降。

TF-IDF的主要思想是，如果某个词在文档中出现的频率高，在其他文档中很少出现，认为这个词具有很好的类别区分能力，适合用来分类

优点在于实现简单高效

缺点在于不能有效反映词的重要程度，没有考虑到特征词的分布情况(特征词的位置，在不同类别文档中的分布)，

参考资料：https://blog.csdn.net/asialee_bird/article/details/81486700

In [6]:
from gensim import models
import numpy as np
 
# 计算词袋模型中的token权重
word_weight =[]
for doc in BoW_corpus:
    for id, freq in doc:
        word_weight.append([my_dictionary[id], freq])
print(word_weight)

[['abc', 1], ['applications', 1], ['computer', 1], ['for', 1], ['human', 1], ['interface', 1], ['lab', 1], ['machine', 1], ['computer', 1], ['of', 2], ['opinion', 1], ['response', 1], ['survey', 1], ['system', 1], ['time', 1], ['user', 1], ['interface', 1], ['system', 1], ['user', 1], ['eps', 1], ['management', 1], ['the', 1], ['human', 1], ['of', 1], ['system', 2], ['eps', 1], ['and', 1], ['engineering', 1], ['testing', 1], ['of', 1], ['response', 1], ['time', 1], ['user', 1], ['error', 1], ['measurement', 1], ['perceived', 1], ['relation', 1], ['to', 1], ['of', 1], ['the', 1], ['binary', 1], ['generation', 1], ['random', 1], ['trees', 1], ['unordered', 1], ['of', 1], ['the', 1], ['trees', 1], ['graph', 1], ['in', 1], ['intersection', 1], ['paths', 1], ['of', 1], ['and', 1], ['trees', 1], ['graph', 1], ['iv', 1], ['minors', 1], ['ordering', 1], ['quasi', 1], ['well', 1], ['widths', 1], ['survey', 1], ['graph', 1], ['minors', 1]]


In [7]:
tfIdf = models.TfidfModel(BoW_corpus, smartirs = 'ntc')

# 查看tf-idf权重
weight_tfidf =[]
for doc in tfIdf[BoW_corpus]:
    for id, freq in doc:
        weight_tfidf.append([my_dictionary[id], np.around(freq, decimals = 3)])
print(weight_tfidf) 

[['abc', 0.393], ['applications', 0.393], ['computer', 0.275], ['for', 0.393], ['human', 0.275], ['interface', 0.275], ['lab', 0.393], ['machine', 0.393], ['computer', 0.363], ['of', 0.231], ['opinion', 0.52], ['response', 0.363], ['survey', 0.363], ['system', 0.272], ['time', 0.363], ['user', 0.272], ['interface', 0.418], ['system', 0.313], ['user', 0.313], ['eps', 0.418], ['management', 0.598], ['the', 0.313], ['human', 0.326], ['of', 0.103], ['system', 0.487], ['eps', 0.326], ['and', 0.326], ['engineering', 0.466], ['testing', 0.466], ['of', 0.088], ['response', 0.278], ['time', 0.278], ['user', 0.208], ['error', 0.398], ['measurement', 0.398], ['perceived', 0.398], ['relation', 0.398], ['to', 0.398], ['of', 0.103], ['the', 0.244], ['binary', 0.466], ['generation', 0.466], ['random', 0.466], ['trees', 0.244], ['unordered', 0.466], ['of', 0.113], ['the', 0.266], ['trees', 0.266], ['graph', 0.266], ['in', 0.508], ['intersection', 0.508], ['paths', 0.508], ['of', 0.087], ['and', 0.273]

# Gensim中N-gram模型使用

Gensim中支持N-gram模型，通过gensim.models.phrases模块

In [ ]:

import gensim.downloader as api
from gensim.models.phrases import Phrases
 
# 读取text8数据集
dataset = api.load("text8")
data =[]
for word in dataset:
    data.append(word)
               
# 使用Phraser模型构建Bigram模型             
bigram_model = Phrases(data, min_count = 3, threshold = 10)
print(bigram_model[data[0]]) 

# 使用Phraser模型构建Trigram模型
trigram_model = Phrases(bigram_model[data], threshold = 10)
print(trigram_model[bigram_model[data[0]]])

# Gemsim中Word2Vec模型使用

Word2Vec是一种Word Embedding方法，将不可计算、非结构化的词转化为可计算、结构化的向量，即将词转换为词向量。

Word2Vec模型基于词袋模型和Skip-gram模型，使用双层神经网络用来训练来重新构建词向量。

参考资料：https://zhuanlan.zhihu.com/p/26306795

In [ ]:
import gensim.downloader as api
from multiprocessing import cpu_count
from gensim.models.word2vec import Word2Vec
 
dataset = api.load("text8")
data =[]
for word in dataset:
    data.append(word)

# 将数据集划分为两部分，一部分训练模型，另一部分更新模型
data_1 = data[:1200]  
data_2 = data[1200:]   
 
# 训练Word2Vec模型
w2v_model = Word2Vec(data_1, min_count = 0, workers = cpu_count())
 
# 展示Word2Vec中的词向量
print(w2v_model['time'])

Word2Vec将词转换为词向量（包含其语义表示），可以通过向量的相似度表示词之间的相似度

In [ ]:
print(w2v_model.most_similar('time'))

Gensim提供了Word2Vec模型的保存和读取接口

In [ ]:
w2v_model.save('Word2VecModel')
model = Word2Vec.load('Word2VecModel')

Gensim支持引入新的数据对已经训练好的Word2Vec模型进行进一步的训练和更新

In [ ]:
w2v_model.build_vocab(data_2, update = True)
w2v_model.train(data_2, total_examples = w2v_model.corpus_count, epochs = w2v_model.iter)
print(w2v_model['time'])